# 2a. Stream to Parquet
--------------------------------------------------------------------

Store the input stream to a set of parquet files. The purpose is to store the input stream to a log of raw events.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

Each batch size (default 1024 records) are stored in a parquet file. The default output is to `/User/examples/model-deployment-with-streaming/data/events-pq`

In [ ]:
import nuclio

In [ ]:
%run config.py

In [ ]:
import os
TARGET_PATH = os.path.join(os.sep, 'v3io', 'users', PARQUET_TARGET_PATH)
input_stream_url = path.join(WEB_API_USERS, STREAM_CONFIGS['generated-stream']['path']) + "@stream2pq"

In [ ]:
# Define function spec
%nuclio config kind = "nuclio"

In [ ]:
%%nuclio cmd -c

python -m pip install pandas
python -m pip install pyarrow

In [ ]:
%%nuclio config
spec.build.baseImage = "mlrun/ml-models"
spec.readinessTimeoutSeconds = 200

In [ ]:
%nuclio mount /User ~/

In [ ]:
# nuclio: start-code

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import datetime

In [ ]:
def init_context(context):
    setattr(context, 'batch', [])
    setattr(context, 'batch_size', int(os.getenv('BATCH_SIZE', 1024)))
    setattr(context, 'batch_count',int(os.getenv('BATCH_COUNT', 0)))
    
    pq_partitions = os.getenv('PQ_PARTITIONS')
    if pq_partitions:
        setattr(context, 'pq_partitions', pq_partitions.split(','))
    else:
        setattr(context, 'pq_partitions', pq_partitions)
    
    setattr(context, 'target_path', os.getenv('TARGET_PATH'))
    os.makedirs(context.target_path, exist_ok=True)

In [ ]:
def handler(context, event):
    if type(event.body) is dict:
        event_dict = event.body
    else:
        event_dict = json.loads(event.body)
        
    context.logger.info_with('Got invoked',
                             trigger_kind=event.trigger.kind,
                             event_body=event_dict)
    
    # add the incoming event to the current batch
    context.batch.append(event_dict)
    
    #check if batch size reached
    if context.batch_size == len(context.batch):
        context.logger.info_with('Writing batch',
                                 batch_count=context.batch_count,
                                 batch_size=len(context.batch))
        write_batch(context)
        context.logger.info_with('Written batch',
                                 batch_count=context.batch_count,
                                 batch_size=len(context.batch))
        
def write_batch(context):
    file_name = str(context.worker_id)+'_'+str(context.batch_count)
    df = pd.DataFrame.from_records(context.batch)
    df.to_parquet(path=os.path.join(context.target_path, file_name), partition_cols=context.pq_partitions)
    # post write cleanup and counter update
    context.batch = []
    context.batch_count += 1

In [ ]:
# nuclio: end-code

## Test Locally

In [ ]:
init_context(context)
#reduce the batch size to 10
context.batch_size = 10

# trigger with 9 events:

nine_events = [b'{"user_id" : 1 , "event_type": "spin"}',
              b'{"user_id" : 2 , "event_type": "spin"}',
              b'{"user_id" : 3 , "event_type": "spin"}',
              b'{"user_id" : 4 , "event_type": "spin"}',
              b'{"user_id" : 5 , "event_type": "spin"}',
              b'{"user_id" : 6 , "event_type": "spin"}',
              b'{"user_id" : 7 , "event_type": "spin"}',
              b'{"user_id" : 8 , "event_type": "spin"}',
              b'{"user_id" : 9 , "event_type": "spin"}']

for e in nine_events:
    event = nuclio.Event(body=e)
    handler(context, event)

In [ ]:
# check whether a parquet has been created
!ls -l {TARGET_PATH}

In [ ]:
# trigger the tenth event which should trigger the creation of the parquet file.
tenth_event = b'{"user_id" : 10 , "event_type": "spin"}'
event = nuclio.Event(body=tenth_event)
handler(context, event)

In [ ]:
# check weather a parquet has been created
!ls -l {TARGET_PATH}

In [ ]:
# cleanup
!rm {TARGET_PATH}/None_0

# MLRUN

In [ ]:
%store -r project_name
%store -r project_path
%store -r artifact_path
%store -r project

In [ ]:
from mlrun import code_to_function

gen_func = code_to_function(name='stream2pq', kind = 'nuclio')
project.set_function(gen_func)
stream2pq = project.func('stream2pq')
stream2pq.set_envs({'TARGET_PATH' : TARGET_PATH, 'BATCH_SIZE': 1024})
stream2pq.add_trigger('stream2pq', nuclio.triggers.V3IOStreamTrigger(url=input_stream_url, access_key=V3IO_ACCESS_KEY,maxWorkers=10))


In [ ]:
#Build image
stream2pq.deploy()